In [1]:
import sys
sys.path.append("../../")

In [2]:
from ptgnn.dataset import RSDataset, BindingAffinityDataset, BaceDataset, Tox21Dataset, OGBDataset

In [3]:
ogb = OGBDataset(split="test", ds_name="hiv")
ogb[0]

Processing...
32901it [04:35, 119.42it/s]
4113it [00:48, 84.59it/s] 
4113it [00:40, 102.43it/s]
Done!


Data(x=[88, 118], edge_index=[2, 252], edge_attr=[252, 80], pos=[88, 6], parallel_node_index=[88], circle_index=[88], y=[1])

In [6]:
bac1 = BaceDataset(split='val')

In [7]:
bac1

BaceDataset(151)

In [8]:
bac1[0]

Data(x=[202, 118], edge_index=[2, 578], edge_attr=[578, 80], pos=[202, 6], parallel_node_index=[202], circle_index=[202], y=[1])

In [11]:
tox1 = Tox21Dataset(split='val')

In [12]:
tox1[0]

Data(x=[40, 118], edge_index=[2, 104], edge_attr=[104, 80], pos=[40, 6], parallel_node_index=[40], circle_index=[40], y=[12])

In [3]:
t1 = RSDataset(split='val')

In [4]:
t1[0]

Data(x=[34, 52], edge_index=[2, 72], edge_attr=[72, 14], pos=[34, 3], bond_distances=[36], bond_distance_index=[2, 36], bond_angles=[60], bond_angle_index=[3, 60], dihedral_angles=[91], dihedral_angle_index=[4, 91], y=[1])

In [5]:
t1

RSDataset(11740)

In [6]:
len(t1)

11740

In [7]:
t2 = RSDataset(split="test")

Processing...
11680it [01:14, 156.32it/s]
Done!


In [8]:
t2[0]

Data(x=[36, 118], edge_index=[2, 104], edge_attr=[104, 80], pos=[36, 6], parallel_node_index=[36], circle_index=[36], y=[1])

In [9]:
len(t2)

11676

In [10]:
t3 = RSDataset(split="train")

Processing...


KeyboardInterrupt: 

In [ ]:
t3[0]

In [ ]:
len(t3)

In [3]:
ba1 = BindingAffinityDataset(split="val")

Processing...
10368it [00:34, 297.81it/s]
Done!


In [4]:
ba1

BindingAffinityDataset(10354)

In [5]:
ba1[0]

Data(x=[46, 118], edge_index=[2, 136], edge_attr=[136, 80], pos=[46, 6], parallel_node_index=[46], circle_index=[46], y=[1])

In [6]:
len(ba1)

10354

In [16]:
import pandas as pd
from chainer_chemistry.dataset.splitters.scaffold_splitter import ScaffoldSplitter
from chainer_chemistry.dataset.splitters.random_splitter import RandomSplitter

In [7]:
df = pd.read_csv("src/bace/raw/bace.csv")

In [11]:
df.head()

,mol,CID,Class,Model,pIC50,MW,AlogP,HBA,HBD,RB,...,PEOE6 (PEOE6),PEOE7 (PEOE7),PEOE8 (PEOE8),PEOE9 (PEOE9),PEOE10 (PEOE10),PEOE11 (PEOE11),PEOE12 (PEOE12),PEOE13 (PEOE13),PEOE14 (PEOE14),canvasUID
0,O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2c...,BACE_1,1,Train,9.154901,431.56979,4.4014,3,2,5,...,53.205711,78.640335,226.85541,107.43491,37.133846,0.000000,7.980170,0.0,0.000000,1
1,Fc1cc(cc(F)c1)C[C@H](NC(=O)[C@@H](N1CC[C@](NC(...,BACE_2,1,Train,8.853872,657.81073,2.6412,5,4,16,...,73.817162,47.171600,365.67694,174.07675,34.923889,7.980170,24.148668,0.0,24.663788,2
2,S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...,BACE_3,1,Train,8.698970,591.74091,2.5499,4,3,11,...,70.365707,47.941147,192.40652,255.75255,23.654478,0.230159,15.879790,0.0,24.663788,3
3,S1(=O)(=O)C[C@@H](Cc2cc(O[C@H](COCC)C(F)(F)F)c...,BACE_4,1,Train,8.698970,591.67828,3.1680,4,3,12,...,56.657166,37.954151,194.35304,202.76335,36.498634,0.980913,8.188327,0.0,26.385181,4
4,S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...,BACE_5,1,Train,8.698970,629.71283,3.5086,3,3,11,...,78.945702,39.361153,179.71288,220.46130,23.654478,0.230159,15.879790,0.0,26.100143,5


In [22]:
splitter = ScaffoldSplitter()
# splitter = RandomSplitter()

In [23]:
df['mol']

0       O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2c...
1       Fc1cc(cc(F)c1)C[C@H](NC(=O)[C@@H](N1CC[C@](NC(...
2       S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...
3       S1(=O)(=O)C[C@@H](Cc2cc(O[C@H](COCC)C(F)(F)F)c...
4       S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...
                              ...                        
1508            Clc1cc2nc(n(c2cc1)C(CC(=O)NCC1CCOCC1)CC)N
1509            Clc1cc2nc(n(c2cc1)C(CC(=O)NCc1ncccc1)CC)N
1510               Brc1cc(ccc1)C1CC1C=1N=C(N)N(C)C(=O)C=1
1511         O=C1N(C)C(=NC(=C1)C1CC1c1cc(ccc1)-c1ccccc1)N
1512                  Clc1cc2nc(n(c2cc1)CCCC(=O)NCC1CC1)N
Name: mol, Length: 1513, dtype: object

In [27]:
splitter.train_valid_test_split(
    df,
    smiles_list=df['mol'].tolist(),
    frac_train=0.7,
    frac_valid=0.1,
    frac_test=0.2,
    seed=0,
    include_chirality=True,
    return_index=True
)

(array([1305, 1312, 1316, ..., 1393, 1426, 1489]),
 array([ 240,  230,   16,   35, 1384, 1375,  465,  596,  613,  657,  658,
         659,  660,  686,  687,  766,  825,  848, 1003, 1009, 1026, 1071,
        1465,  944,  972, 1116,   30,   47,   96,  645,   65,  190,  206,
         210,  245,  246,  247, 1192,  237,  205, 1261,  743, 1145, 1477,
          97,  381, 1281,  361,  197, 1457, 1460, 1461, 1482, 1492, 1505,
        1506,  738,  497,  252,  260,  279,  294,  363,  364,  430,  458,
         459,  571,  673,  681,  682,  818, 1175,  338, 1284, 1336,  889,
           9, 1439, 1319,  212,  772, 1021, 1435, 1130,   91,  178, 1135,
        1200, 1376, 1381, 1414,  539,  425,  722,  750,  751,  842,  873,
         963,  990,    1,  404, 1001, 1214, 1286,  677,  680, 1230,  651,
        1473,  103, 1047, 1352, 1472,  995, 1068, 1353, 1425, 1438,  536,
         850, 1014, 1252, 1341,  874,  785,  463,  589, 1497,  590,  906,
         351, 1043,  783,  923, 1143, 1185, 1368,  644,  400,